In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# `AvoidReaver`
- SAIDA_RL에서 작성한 document : https://teamsaida.github.io/SAIDA_RL/AvoidReavers/ 

## #1. import `AvoidReavers`

- `AvoidReaver` 클래스를 import 해서 env를 만들면 학습시킬 수 있는 환경 객체가 생성된다.

In [3]:
from saida_gym.starcraft.avoidReavers import AvoidReavers

In [7]:
env = AvoidReavers(move_angle=30, move_dist=2, frames_per_step=16, verbose=0, action_type=0, no_gui=False, \
                   bot_runner=r"..\SAIDA_RL\cpp\Release\SAIDA\SAIDA.exe")
env

Initialize...
Shared Memory create 
SAIDA_AR6840 Shared memory found.


- env 객체를 생성하기만 해도 starcraft 환경이 켜짐. 


- `python/saida_gym/envs/SAIDAGym.py` 에서 `shm.SharedMemory()` 로 메모리를 공유하면서 바로 켜지는 것으로 보임.

---
## #2. `env`는 어떤 속성들을 가지고 있나?


In [8]:
env.__dict__.keys()

dict_keys(['verbose', 'action_type', 'map_version', 'frames_per_step', 'move_angle', 'move_dist', 'conn', 'message', 'common_message', 'init_req_msg', 'init_res_msg', 'reset_req_msg', 'reset_res_msg', 'step_req_msg', 'step_res_msg', 'close_req_msg', 'state', 'action_space', 'reward', 'done', 'info'])

- 하나씩 확인해보자.

### 1) 학습과 관련된 parameters

In [10]:
print("verbose:", env.verbose)
print("action_type:", env.action_type)
print("map_version:", env.map_version)
print("frames_per_step:", env.frames_per_step)
print("move_angle:", env.move_angle)
print("move_dist:", env.move_dist)
print("action_space:", env.action_space)

verbose: 0
action_type: 0
map_version: 0
frames_per_step: 16
move_angle: 30
move_dist: 2
action_space: Discrete(13)


#### # `verbose`
- 출력되는 로그의 양 조정

#### # `action_type`
- `0` : discrete (output : descrete한 N개의 action_space 중 하나의 `integer` (0~N 중 마지막 숫자인 N은 공격))
- `1` : continuous, xy좌표 (output : `[x좌표, y좌표, action_종류(0:이동, 1:공격)]`)
- `2` : continuous, 극좌표 (output : `[theta, radian, action_종류(0:이동, 1:공격)]`)
    
#### # `map_version` 
- `0` : 1 vulture versus 1 zealot, no terrain
- `1` : 1 vulture versus 2 zealot, no terrain	
- `2` : 1 vulture versus 1 zealot, with terrain
- `3` : 1 vulture versus 2 zealot, with terrain

#### # `frames_per_step`
- 몇 frame마다 하나의 action을 결정할지에 대한 step을 결정

#### # `move_angle`
- 가장 작은 단위의 움직임 각도 단위
- `move_angle = 15` 이면 agent는 15도 단위, 즉 360/15 = 24방향으로 이동가능

#### # `move_distance` 
- 한 action에 이동할 거리
- 1 unit = 32px 

#### # `action_space` 
- `move_angle`로 나뉘어진 action 공간
- `move_angle = 15` 이면 `action_space`는 24

#### # `observation_space`
- 각 agent (자신과 적 포함) 에게서 관찰할 수 있는 모든 정보 종류의 수

### 2) starcraft server와의 connection과 관련된 parameters

In [11]:
print("conn:", env.conn)
print("message:", env.message)  # protobuf file
print("common_message:", env.common_message)  # protobuf file

conn: <saida_gym.envs.conn.shm.SharedMemory object at 0x000001BB396E06D8>
message: <module 'saida_gym.envs.protobuf.common_pb2' from 'C:\\Users\\Jeina\\SAIDA_RL\\python\\saida_gym\\envs\\protobuf\\common_pb2.py'>
common_message: <module 'saida_gym.envs.protobuf.common_pb2' from 'C:\\Users\\Jeina\\SAIDA_RL\\python\\saida_gym\\envs\\protobuf\\common_pb2.py'>


- 이 친구들은 connection과 연관이 있음 


- python 코드에서 cpp 코드로 넘어가는 부분이기도 함


- python과 cpp 간에 데이터 전송을 protobuf 라는 데이터 구조로 진행하는 것으로 보임, 완벽하게 이해는 x  

### 3) `state`

- 맨 처음 `env` 객체를 만든 후 state를 호출하면 아무것도 없음

In [12]:
env.state

- `env`를 한 번 reset 해줘야 env에 최초 state가 담김

In [16]:
state = env.reset()
state

my_unit {
  unit_type: "Terran_Dropship"
  hp: 150
  pos_x: 35
  pos_y: 35
  angle: 0.9817477042468103
}
en_unit {
  unit_type: "Protoss_Reaver"
  hp: 100
  shield: 80
  cooldown: 11
  pos_x: 96
  pos_y: 256
  angle: 5.522330836388308
  accelerating: true
}
en_unit {
  unit_type: "Protoss_Reaver"
  hp: 100
  shield: 80
  cooldown: 11
  pos_x: 192
  pos_y: 192
  angle: 3.902447124381071
  accelerating: true
}
en_unit {
  unit_type: "Protoss_Reaver"
  hp: 100
  shield: 80
  cooldown: 11
  pos_x: 256
  pos_y: 64
  angle: 0.6872233929727672
  accelerating: true
}

- en_unit은 세 마리의 Reavers를 나타냄

### 4) `reward`, `done`, `info`

In [17]:
env.reward

0

- agent의 보상값 (zealot을 죽이면 양의 보상을 받을 것으로 예상)

In [18]:
env.done

0

- episode가 끝났는지에 대한 boolean값

- vulture나 zealot 중 하나가 죽으면 끝날 것으로 예상

In [19]:
env.info

{}